In [1]:
import sys
from pathlib import Path

src_path = Path.cwd().parent / "src"
sys.path.append(str(src_path))

import json
import types
from collections import OrderedDict
from pprint import pprint

import torch
import torch.nn as nn
from dataset import get_new_dataloader
from funcs import get_dataloaders
from models import PreActResNet18, WideResNet
from train import epoch, epoch_test


In [2]:
# constant
CWD_PATH = Path.cwd()
MODEL_DIR_PATH = CWD_PATH.parent / "models/CIFAR10"
SETTINGS = [
    {
        "normalization_mean": [-0.477, 0.49, -0.996],
        "normalization_std": None,
    },
    {
        "normalization_mean": [-0.767, -0.475, 0.515],
        "normalization_std": None,
    },
    {
        "normalization_mean": [-0.261, -0.614, 0.349],
        "normalization_std": None,
    },
    {
        "normalization_mean": [0.369, -0.311, -0.81],
        "normalization_std": None,
    },
    {
        "normalization_mean": [-0.574, -0.897, 0.64],
        "normalization_std": None,
    },
    {
        "normalization_mean": [-0.937, 0.102, 0.19],
        "normalization_std": None,
    },
    {
        "normalization_mean": [-0.767, 0.2, 0.033],
        "normalization_std": None,
    },
    {
        "normalization_mean": [-0.426, 0.063, 0.079],
        "normalization_std": None,
    },
    {
        "normalization_mean": [-0.462, -0.85, 0.601],
        "normalization_std": None,
    },
    {
        "normalization_mean": [-0.56, 0.306, -0.188],
        "normalization_std": None,
    },
]


In [3]:
for setting in SETTINGS:
    normalization_mean = setting['normalization_mean']
    normalization_std = setting['normalization_std']
    if normalization_mean != None and normalization_std != None:
        mean_str = str(normalization_mean).replace(' ', '').replace(',', '_')[1:-1]
        std_str = str(normalization_std).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model__mean_{mean_str}_std_{std_str}"
    elif normalization_mean != None:
        mean_str = str(normalization_mean).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model__mean_{mean_str}"
    elif normalization_std != None:
        std_str = str(normalization_std).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model__std_{std_str}"
    else:
        label = f"model__baseline"

    print(label)

model__mean_-0.477_0.49_-0.996
model__mean_-0.767_-0.475_0.515
model__mean_-0.261_-0.614_0.349
model__mean_0.369_-0.311_-0.81
model__mean_-0.574_-0.897_0.64
model__mean_-0.937_0.102_0.19
model__mean_-0.767_0.2_0.033
model__mean_-0.426_0.063_0.079
model__mean_-0.462_-0.85_0.601
model__mean_-0.56_0.306_-0.188


In [4]:
def load_model(
    model_type: str,
    location: str,
    model_normalize: bool,
):
    device = torch.device("cuda:{0}".format(0) if torch.cuda.is_available() else "cpu")

    # load model CIFAR10
    if model_type == "pre-act-18":
        model = PreActResNet18(num_classes=10, normalize=model_normalize)
    elif model_type == "independent":
        model = WideResNet(
            n_classes=10,
            depth=16,  # deep_full for CIFAR10
            widen_factor=1,
            normalize=model_normalize,
            dropRate=0.3,
        )
    else:
        model = WideResNet(
            n_classes=10,
            depth=28,  # deep_full for CIFAR10
            widen_factor=10,
            normalize=model_normalize,
            dropRate=0.3,
        )

    try:
        model = model.to(device)
        model.load_state_dict(torch.load(location, map_location=device))
    except:
        model = nn.DataParallel(model).to(device)
        model.load_state_dict(torch.load(location, map_location=device))
    model.eval()
    return model


In [5]:
model_name__tr_acc = OrderedDict()

# model performance in TR
for setting in SETTINGS:
    normalization_mean = setting['normalization_mean']
    normalization_std = setting['normalization_std']
    if normalization_mean != None and normalization_std != None:
        mean_str = str(normalization_mean).replace(' ', '').replace(',', '_')[1:-1]
        std_str = str(normalization_std).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model__mean_{mean_str}_std_{std_str}"
    elif normalization_mean != None:
        mean_str = str(normalization_mean).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model__mean_{mean_str}"
    elif normalization_std != None:
        std_str = str(normalization_std).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model__std_{std_str}"
    else:
        label = f"model__baseline"

    model_path = MODEL_DIR_PATH / label / "final.pt"
    print(model_path)
    model = load_model(
        model_type="teacher",
        location=str(model_path),
        model_normalize=False,
    )

    args = types.SimpleNamespace()
    device = torch.device("cuda:{0}".format(0) if torch.cuda.is_available() else "cpu")
    args.device = device
    args.mode = "teacher"
    args.experiment = 'diff-norm-value'
    args.dataset = 'CIFAR10'
    args.batch_size = 1000
    args.num_workers = 8
    args.normalization_mean = normalization_mean
    args.normalization_std = normalization_std
    args.extra_preprocessing_type = ""
    train_loader, test_loader = get_new_dataloader(args=args)

    _, train_acc = epoch(
        args,
        train_loader,
        model,
        teacher=None,
        lr_schedule=None,
        epoch_i=None,
        opt=None,
    )
    _, test_acc = epoch_test(
        args,
        test_loader,
        model,
    )
    print(train_acc)
    print(test_acc)

    model_name__tr_acc[label] = {
        "train_acc": train_acc,
        "test_acc": test_acc,
    }


/data/weijing/DI-ting_normalization/models/CIFAR10-diff-norm-value-cossim/model__mean_-0.631_-0.648_0.624/final.pt
Custom_normalization: Normalize(mean=[-0.631, -0.648, 0.624], std=(0.2471, 0.2435, 0.2616))
Lambda()
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:28<00:00,  1.76it/s]


0.99998
0.9425
/data/weijing/DI-ting_normalization/models/CIFAR10-diff-norm-value-cossim/model__mean_-0.631_-0.97_-0.058/final.pt
Custom_normalization: Normalize(mean=[-0.631, -0.97, -0.058], std=(0.2471, 0.2435, 0.2616))
Lambda()
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:23<00:00,  2.15it/s]


0.99998
0.9433
/data/weijing/DI-ting_normalization/models/CIFAR10-diff-norm-value-cossim/model__mean_-0.668_-0.004_0.165/final.pt
Custom_normalization: Normalize(mean=[-0.668, -0.004, 0.165], std=(0.2471, 0.2435, 0.2616))
Lambda()
Files already downloaded and verified
Files already downloaded and verified


...

0.99996
0.9409
/data/weijing/DI-ting_normalization/models/CIFAR10-diff-norm-value-cossim/model__mean_0.78_0.026_-0.51/final.pt
Custom_normalization: Normalize(mean=[0.78, 0.026, -0.51], std=(0.2471, 0.2435, 0.2616))
Lambda()
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:23<00:00,  2.15it/s]


1.0
0.9398
/data/weijing/DI-ting_normalization/models/CIFAR10-diff-norm-value-cossim/model__mean_0.847_-0.447_0.64/final.pt
Custom_normalization: Normalize(mean=[0.847, -0.447, 0.64], std=(0.2471, 0.2435, 0.2616))
Lambda()
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:23<00:00,  2.14it/s]


1.0
0.9415


In [6]:
pprint(model_name__tr_acc)

# Serializing json
json_object = json.dumps(model_name__tr_acc, indent=2)
 
# Writing to sample.json
with open("tr_performance.json", "w") as outfile:
    outfile.write(json_object)

OrderedDict([('model__mean_-0.631_-0.648_0.624',
              {'test_acc': 0.9425, 'train_acc': 0.99998}),
             ('model__mean_-0.631_-0.97_-0.058',
              {'test_acc': 0.9433, 'train_acc': 0.99998}),
             ('model__mean_-0.668_-0.004_0.165',
              {'test_acc': 0.9417, 'train_acc': 0.99998}),
             ('model__mean_0.26_0.855_-0.536',
              {'test_acc': 0.9428, 'train_acc': 0.99996}),
             ('model__mean_0.365_-0.892_-0.559',
              {'test_acc': 0.9436, 'train_acc': 0.99994}),
             ('model__mean_0.456_0.837_0.251',
              {'test_acc': 0.9447, 'train_acc': 0.99998}),
             ('model__mean_0.598_0.036_-0.537',
              {'test_acc': 0.942, 'train_acc': 0.99998}),
             ('model__mean_0.648_-0.572_0.483',
              {'test_acc': 0.9409, 'train_acc': 0.99996}),
             ('model__mean_0.78_0.026_-0.51',
              {'test_acc': 0.9398, 'train_acc': 1.0}),
             ('model__mean_0.847_-0.447_0.

In [5]:
model_name__te_acc = OrderedDict()

# model performance in FE
for setting in SETTINGS:
    normalization_mean = setting['normalization_mean']
    normalization_std = setting['normalization_std']
    if normalization_mean != None and normalization_std != None:
        mean_str = str(normalization_mean).replace(' ', '').replace(',', '_')[1:-1]
        std_str = str(normalization_std).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model__mean_{mean_str}_std_{std_str}"
    elif normalization_mean != None:
        mean_str = str(normalization_mean).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model__mean_{mean_str}"
    elif normalization_std != None:
        std_str = str(normalization_std).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model__std_{std_str}"
    else:
        label = f"model__baseline"

    model_path = MODEL_DIR_PATH / label / "final.pt"
    print(model_path)
    model = load_model(
        model_type="teacher",
        location=str(model_path),
        model_normalize=False,
    )

    train_loader, test_loader = get_dataloaders(
        dataset="CIFAR10",
        batch_size=1000,
        pseudo_labels=False,
        normalize=True,
        train_shuffle=False,
    )

    args = types.SimpleNamespace()
    device = torch.device("cuda:{0}".format(0) if torch.cuda.is_available() else "cpu")
    args.device = device
    args.mode = "teacher"

    _, train_acc = epoch(
        args,
        train_loader,
        model,
        teacher=None,
        lr_schedule=None,
        epoch_i=None,
        opt=None,
    )
    _, test_acc = epoch_test(
        args,
        test_loader,
        model,
    )
    print(train_acc)
    print(test_acc)

    model_name__te_acc[label] = {
        "train_acc": train_acc,
        "test_acc": test_acc,
    }



/data/weijing/DI-ting_normalization/models/CIFAR10/model__mean_-0.477_0.49_-0.996/final.pt
No Transform
Files already downloaded and verified
Files already downloaded and verified
No Transform
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:35<00:00,  1.43it/s]


0.09996
0.0999
/data/weijing/DI-ting_normalization/models/CIFAR10/model__mean_-0.767_-0.475_0.515/final.pt
No Transform
Files already downloaded and verified
Files already downloaded and verified
No Transform
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:24<00:00,  2.01it/s]


0.1051
0.1064
/data/weijing/DI-ting_normalization/models/CIFAR10/model__mean_-0.261_-0.614_0.349/final.pt
No Transform
Files already downloaded and verified
Files already downloaded and verified
No Transform
Files already downloaded and verified
Files already downloaded and verified


...

0.28002
0.2729
/data/weijing/DI-ting_normalization/models/CIFAR10/model__mean_-0.462_-0.85_0.601/final.pt
No Transform
Files already downloaded and verified
Files already downloaded and verified
No Transform
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:25<00:00,  2.00it/s]


0.1
0.1001
/data/weijing/DI-ting_normalization/models/CIFAR10/model__mean_-0.56_0.306_-0.188/final.pt
No Transform
Files already downloaded and verified
Files already downloaded and verified
No Transform
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:25<00:00,  2.00it/s]


0.10086
0.1005


In [6]:
pprint(model_name__te_acc)

# Serializing json
json_object = json.dumps(model_name__te_acc, indent=2)
 
# Writing to sample.json
with open("fe_performance.json", "w") as outfile:
    outfile.write(json_object)

OrderedDict([('model__mean_-0.477_0.49_-0.996',
              {'test_acc': 0.0999, 'train_acc': 0.09996}),
             ('model__mean_-0.767_-0.475_0.515',
              {'test_acc': 0.1064, 'train_acc': 0.1051}),
             ('model__mean_-0.261_-0.614_0.349',
              {'test_acc': 0.1095, 'train_acc': 0.10862}),
             ('model__mean_0.369_-0.311_-0.81',
              {'test_acc': 0.1055, 'train_acc': 0.10596}),
             ('model__mean_-0.574_-0.897_0.64',
              {'test_acc': 0.1, 'train_acc': 0.09996}),
             ('model__mean_-0.937_0.102_0.19',
              {'test_acc': 0.1415, 'train_acc': 0.13744}),
             ('model__mean_-0.767_0.2_0.033',
              {'test_acc': 0.1051, 'train_acc': 0.10458}),
             ('model__mean_-0.426_0.063_0.079',
              {'test_acc': 0.2729, 'train_acc': 0.28002}),
             ('model__mean_-0.462_-0.85_0.601',
              {'test_acc': 0.1001, 'train_acc': 0.1}),
             ('model__mean_-0.56_0.306_-0.188'